# Algorytmy przeszukiwania Dijkstra, A*. Lokalne przeszukiwanie i Tabu Search
Natalia Malinowska 272669

##  Użyte biblioteki
W zadaniu wykorzystane zostały następujące standardowe biblioteki Pythona:

```
math
datetime
os
json
csv
bisect
random
heapq
```

In [3]:
import math
from datetime import datetime
import os
import json
import csv
import bisect
import random
import heapq

In [ ]:
def time_to_minutes(time_str):
    parts = list(map(int, time_str.split(':')))
    h = parts[0]
    m = parts[1]
    
    return (h % 24) * 60 + m


def minutes_to_time(minutes):
    h = minutes // 60 % 24
    m = minutes % 60
    return f"{h:02d}:{m:02d}"

# Struktury danych

W zadaniu wykorzystana została autorska struktura grafu, znajdująca się w pliku `graph.py`. Za jej pomocą modelowane są dane z dostarczonego pliku csv.

## Node
Każdy węzeł ma następujące atrybuty:
- `name` – unikalna nazwa węzła,
- `lat, lon` – współrzędne geograficzne węzła,
- `outgoing_edges` – lista wychodzących krawędzi (posortowana według czasu odjazdu).


In [ ]:
class Node:
    def __init__(self, name, lat, lon):
        self.name = name
        self.lat = lat
        self.lon = lon
        self.outgoing_edges = []
        
    # def add_outgoing_edge(self, edge):
    #     self.outgoing_edges.append(edge)
    
    def add_outgoing_edge(self, edge):
        bisect.insort(self.outgoing_edges, edge, key=lambda x: x.dep_minutes)
        
    def get_outgoing_edges(self):
        return self.outgoing_edges
        
    def __eq__(self, other):
        if isinstance(other, Node):
            return self.name == other.name
        return False
    
    #arbitrary but necessary for heapq
    def __lt__(self, other):
        return self.name < other.name

    
    def __hash__(self):
        return hash(self.name)
        
    def __str__(self):
        return f"Node({self.name}, lat={self.lat}, lon={self.lon})"
    
    def __repr__(self):
        return f"Node(name='{self.name}', lat={self.lat}, lon={self.lon})"

## Edge
Każda krawędź reprezentuje połączenie między dwoma węzłami i zawiera:
- `start` – węzeł początkowy,
- `end` – węzeł końcowy,
- `line` – oznaczenie linii (np. numer trasy transportowej),
- `dep_time, arr_time` – czas odjazdu i przyjazdu,
- `travel_time` – czas podróży w minutach,
- `dep_minutes, arr_minutes` – czas odjazdu i przyjazdu przeliczony na minuty od początku dnia.

Potrzebne były również funkcje pomocnicze do zarządzania czasem.

In [7]:
class Edge:
    def __init__(self, start, end, line, dep_time, arr_time, travel_time):
        self.start = start
        self.end = end
        self.line = line
        self.dep_time = dep_time
        self.arr_time = arr_time
        self.travel_time = travel_time
        self.dep_minutes = time_to_minutes(dep_time)
        self.arr_minutes = time_to_minutes(arr_time)
        
    def __eq__(self, other):
        if isinstance(other, Edge):
            return self.start == other.start and self.end == other.end and self.line == other.line and self.dep_time == other.dep_time and self.arr_time == other.arr_time and self.travel_time == other.travel_time
        return False
    
    def __hash__(self):
        return hash((self.start, self.end, self.line, self.dep_time, self.arr_time, self.travel_time)) 
        
    def __str__(self):
        return f"Edge({self.start}, {self.end}, line={self.line}, dep_time={self.dep_time}, arr_time={self.arr_time}, travel_time={self.travel_time})"
    
    def __repr__(self):
        return (
            f"Edge(start={self.start.name}, end={self.end.name}, "
            f"line='{self.line}', dep_time={self.dep_time}, arr_time={self.arr_time}, travel_time={self.travel_time})"
        )


## Graph
Obiekt grafu przechowuje węzły i krawędzie oraz umożliwia:
- Dodawanie nowych węzłów (`add_node`),
- Dodawanie nowych krawędzi (`add_edge`),
- Pobieranie węzła po nazwie (`get_node`),
- Serializację do JSON (`to_json`) oraz deserializację z JSON (`from_json`).

In [8]:

class Graph:
    def __init__(self, nodes, edges):
        self.nodes = nodes if nodes is not None else []
        self.edges = edges if edges is not None else []
        
    def add_node(self, node):
        self.nodes.append(node)
        
    def add_edge(self, edge):
        self.edges.append(edge)
        
    def get_node(self, name):
        for node in self.nodes:
            if node.name == name:
                return node
        return None
        
    def get_nodes(self):
        return self.nodes
    
    def get_edges(self):
        return self.edges
    
    def to_json(self, filename):
        graph_data = {
            'nodes': [
                {
                    'name': node.name,
                    'lat': node.lat,
                    'lon': node.lon,
                } for node in self.nodes
            ],
            'edges': [
                {
                    'start': edge.start.name,
                    'end': edge.end.name,
                    'line': edge.line,
                    'dep_time': edge.dep_time,
                    'arr_time': edge.arr_time,
                    'dep_minutes': edge.dep_minutes,
                    'arr_minutes': edge.arr_minutes, 
                    'travel_time': edge.travel_time
                } for edge in self.edges
            ]
        }
        with open(filename, 'w') as f:
            json.dump(graph_data, f, indent=2)
    
    @classmethod
    def from_json(cls, filename):
        with open(filename) as f:
            graph_data = json.load(f)
        
        nodes = [
            Node(node['name'], node['lat'], node['lon'])
            for node in graph_data['nodes']
        ]
        
        node_dict = {node.name: node for node in nodes}
        
        edges = []
        for edge_data in graph_data['edges']:
            start_node = node_dict[edge_data['start']]
            end_node = node_dict[edge_data['end']]
            
            edge = Edge(
                start_node,
                end_node,
                edge_data['line'],
                edge_data['dep_time'],
                edge_data['arr_time'],
                edge_data['travel_time']
            )
            edge.dep_minutes = edge_data['dep_minutes']
            edge.arr_minutes = edge_data['arr_minutes']
            
            edges.append(edge)
            start_node.add_outgoing_edge(edge)
        
        return cls(nodes, edges)

# Funkcje pomocnicze

W zadaniu wykorzystuję szereg funkcji pomocniczych, znajdujących się w pliku `utils.py`. Oto najważniejsze z nich:

## get_graph()

Umożliwia zbudowanie grafu z danych w csv oraz jego serializację do formatu json celem szybszego działania pomiędzy wywołaniami.

In [ ]:
GRAPH_JSON_FILE = "graph.json"

def get_graph():
    if os.path.exists(GRAPH_JSON_FILE):
        print("Loading graph from JSON cache...")
        return Graph.from_json(GRAPH_JSON_FILE)

    print("Generating graph from CSV...")
    unique_nodes = {}
    unique_edges = set()
    start_time = datetime.now()

    with open('connection_graph.csv', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for data_row in reader:
            try:
                start = data_row["start_stop"]
                end = data_row["end_stop"]
                line = data_row["line"]
                dep_time = data_row["departure_time"]
                arr_time = data_row["arrival_time"]
                start_stop_lat = float(data_row["start_stop_lat"])
                start_stop_lon = float(data_row["start_stop_lon"])
                end_stop_lat = float(data_row["end_stop_lat"])
                end_stop_lon = float(data_row["end_stop_lon"])

                dep_h, dep_m, dep_s = map(int, dep_time.split(":"))
                arr_h, arr_m, arr_s = map(int, arr_time.split(":"))
                dep_mins = dep_h * 60 + dep_m
                arr_mins = arr_h * 60 + arr_m
                travel_time = abs(arr_mins - dep_mins)
                
                if dep_h >= 24:
                    dep_time = f"{(dep_h-24):02d}:{(dep_m):02d}:00"
                
                if arr_h >= 24:
                    arr_time = f"{(arr_h-24):02d}:{(arr_m):02d}:00"

                if start not in unique_nodes:
                    unique_nodes[start] = Node(start, start_stop_lat, start_stop_lon)
                if end not in unique_nodes:
                    unique_nodes[end] = Node(end, end_stop_lat, end_stop_lon)
                
                nodeA = unique_nodes[start]
                nodeB = unique_nodes[end]
                
                edge = Edge(nodeA, nodeB, line, dep_time, arr_time, travel_time)
                unique_edges.add(edge)
                
            except Exception as e:
                print(f"Error in line {data_row}: {e}")

    for edge in unique_edges:
        edge.start.add_outgoing_edge(edge)
        
    end_time = datetime.now()
    print(f"Graph initialization execution time: {end_time - start_time} seconds")
    
    for node in unique_nodes.values():
        node.outgoing_edges.sort(key=lambda e: tuple(map(int, e.dep_time.split(":"))))

    graph = Graph(list(unique_nodes.values()), list(unique_edges))

    graph.to_json(GRAPH_JSON_FILE)
    
    return graph

## reconstruct_path() oraz print_path()
### format_time(), calculate_total_travel_time() oraz log()
Metody pomocne przy rekonstrukcji ścieżek wyznaczonych przez algorytm oraz ładnym i czytelnym ich sformatowaniu.

In [ ]:
def format_time(timestamp):
    return timestamp.strftime("%Y-%m-%d %H:%M:%S.%f")[:-4]

def calculate_total_travel_time(start_time, final_arrival_time):
    if final_arrival_time < start_time:
        final_arrival_time += 24 * 60
    
    return final_arrival_time - start_time

def log(message):
    print(f"[{format_time(datetime.now())}] {message}")

def reconstruct_path(previous, starting_stop, destination_stop):
    start_time = datetime.now()
    path = []
    current = destination_stop
    final_arrival_time = None
    while current != starting_stop:
        prev_node, edge_used, line_used = previous.get(current, (None, None, None))
        if prev_node is None:
            return None
        print(f'{current} <- {prev_node} ({edge_used.dep_time})')
        path.append((prev_node, edge_used, current, line_used))
        current = prev_node
        
        if final_arrival_time is None:
            final_arrival_time = time_to_minutes(edge_used.arr_time)
    path.reverse()
    
    stop_algorithm_time = datetime.now()
    log(f"Path reconstruction execution time: {stop_algorithm_time - start_time} seconds")
    return path, final_arrival_time



def print_path(path, starting_stop_name, start_time, total_travel_time=0):
    if not path:
        print("No complete path found!")
        return

    print(f"\nShortest path from {starting_stop_name} at {start_time}:")
    print(f"Start at {path[0][0].name} (Time: {start_time})")

    current_line = None
    last_arrival_time = start_time

    for prev_node, edge, current_node, line in path:
        if line != current_line:
            print(f"  → Change to line {line} at {prev_node.name}")
            current_line = line

        print(f"    → Depart at {edge.dep_time} from {prev_node.name}")
        print(f"    → Arrive at {current_node.name} at {edge.arr_time} ({edge.travel_time} mins)")

        last_arrival_time = edge.arr_time

    start_minutes = time_to_minutes(start_time)
    arrival_minutes = time_to_minutes(last_arrival_time)
    total_travel_time = arrival_minutes - start_minutes

    print(f"\nTotal travel time: {total_travel_time} minutes")

## get_user_input() oraz main()

Te metody posłużyły, aby utworzyć CLI dla użytkownika.

In [ ]:
def get_user_input():
    print("Choose an algorithm:")
    print("1. Dijkstra")
    print("2. A*")
    print("3. Tabu Search")
    print("4. debug")
    
    choice = input("Enter the number of the algorithm: ").strip()
    
    if choice in ['1', '2']:
        start_stop = input("Enter the start stop: ").strip()
        end_stop = input("Enter the end stop: ").strip()
        start_time = input("Enter the start time (HH:MM): ").strip()
        criteria = input("Enter criteria (t for time, p for preference): ").strip()
        
        if choice == '2':  # A*
            heuristic = input("Enter heuristic (euclidean, manhattan, haversine): ").strip()
            return choice, start_stop, end_stop, start_time, criteria, heuristic
        
        return choice, start_stop, end_stop, start_time, criteria
    elif choice == '3':
        start_stop = input("Enter the start stop: ").strip()
        stop_list = input("Enter a list of stops separated by semicolons: ").strip().split(';')
        start_time = input("Enter the start time (HH:MM): ").strip()
        criteria = input("Enter criteria (t for time, p for preference): ").strip()
        
        return choice, start_stop, stop_list, start_time, criteria
    else:
        return choice, None, None, None, None, None
    

def main():
    print("Initializing graph...")
    graph = get_graph()
    print(f"Graph loaded with {len(graph.nodes)} nodes and {len(graph.edges)} edges.")
    
    user_input = get_user_input()
    
    if user_input[0] == '1':
        path, total_time = find_dijkstra_path(graph, user_input[1], user_input[2], user_input[3], user_input[4])
        print(f"Dijkstra Path: {path}, Total time: {total_time}")
        print_path(path, user_input[1], user_input[3], total_time)
    
    elif user_input[0] == '2':
        path, total_time, _ = find_a_star_path(graph, user_input[1], user_input[2], user_input[3], user_input[4], user_input[5])
        print(f"A* Path: {path}, Total time: {total_time}")
        print_path(path, user_input[1], user_input[3], total_time)
    
    elif user_input[0] == '3':
        solution = tabu_search(graph, user_input[1], user_input[2], user_input[3], user_input[4])
        print(f"Tabu Search Solution: {solution[0]}, Cost: {solution[1]}")
        print(f"Path: {solution[2]}")
        print_path(solution[2], user_input[1], user_input[3])
        
    else: #debug!!
        # path, total_time = find_dijkstra_path(graph, "PL. GRUNWALDZKI", "Wrocławski Park Przemysłowy", "14:40", 'p')
        # path, total_time = find_dijkstra_path(graph, "most Grunwaldzki", "Wrocławski Park Przemysłowy", "14:41", 'p')
        path, total_time = find_dijkstra_path(graph, "PL. GRUNWALDZKI", "Wrocławski Park Przemysłowy", "14:40", 't')
        print(f"Dijkstra Path: {path}, Total time: {total_time}")
        #print_path(path, "PL. GRUNWALDZKI", "14:40", total_time)
        print_path(path, "PL. GRUNWALDZKI", "14:40", total_time)
        path, total_time = find_dijkstra_path(graph, "PL. GRUNWALDZKI", "Wrocławski Park Przemysłowy", "14:40", 'p')
        print_path(path, "PL. GRUNWALDZKI", "14:40", total_time)


# Zadanie 1.

## a) Algorytm Dijkstry

### Opis teoretyczny metody
Algorytm Dijkstry jest jednym z najbardziej znanych algorytmów znajdowania najkrótszej ścieżki w grafie o dodatnich wagach. Działa na zasadzie stopniowego rozszerzania zbioru wierzchołków, dla których najkrótsza ścieżka od węzła początkowego została już znaleziona. Wykorzystuje kolejkę priorytetową do wyboru wierzchołka o najniższym koszcie przejścia.

### Przykładowe zastosowania
nawigacja GPS, transport publiczny, optymalizacja tras dla sieci komputerowych

### Wprowadzone modyfikacje
Poniżej wypisano szereg modyfikacji wprowadzonych do algorytmu. Nie wszystkie z nich okazały się działać poprawnie.
- wprowadzenie kolejki priorytetowej
- śledzenie najwcześniejszego znalezionego czasu dotarcia na przystanek celem optymalizacji czasu wykonywania
- wprowadzenie liniowej kary za przesiadki
- śledzenie ilości przesiadek
- próba priorytetyzowania tej samej linii odjeżdzającej o tej samej godzienie z badanego przystanku
- różne kombinacje wartości w kolejce priorytetowej

### Materiały dodatkowe
- wizualizacja Dijkstry [https://www.cs.usfca.edu/~galles/visualization/Dijkstra.html]
- Dijkstra z kolejką priorytetową [https://www.geeksforgeeks.org/dijkstras-shortest-path-algorithm-using-priority_queue-stl/]
- [https://en.wikipedia.org/wiki/Dijkstra's_algorithm]

### Wykorzystane biblioteki i funkcje pomocnicze
- datetime do mierzenia czasu
- heapq do kolejki priorytetowej



In [ ]:
def find_dijkstra_path(graph, starting_stop_name, destination_stop_name, start_time, criteria):
    start_algorithm_time = datetime.now()
    starting_stop = graph.get_node(starting_stop_name)
    destination_stop = graph.get_node(destination_stop_name)
    
    if not starting_stop or not destination_stop:
        print("Error: Invalid start or destination stop")
        return None, None
    
    start_total = time_to_minutes(start_time)

    distance = {node: float('inf') for node in graph.get_nodes()}
    previous = {node: (None, None, None) for node in graph.get_nodes()}
    earliest_arrival = {node: float('inf') for node in graph.get_nodes()}
    
    distance[starting_stop] = 0
    earliest_arrival[starting_stop] = start_total
    
    visited = set()
    
    #priority queue: (transfer_count, earliest_arrival, total_cost, current_stop, current_line)
    priority_queue = [(0, 0, start_total, starting_stop, None)]
    heapq.heapify(priority_queue)
    
    while priority_queue:
        # current_transfers, current_cost, current_stop, current_line, current_time = heapq.heappop(priority_queue)
        current_transfers, current_cost, current_time, current_stop, current_line = heapq.heappop(priority_queue)
        
        if current_stop in visited and distance[current_stop] < current_cost:
            continue
        
        visited.add(current_stop)
        
        #skip if we've already found a better path to this node
        if current_time > earliest_arrival[current_stop]:
            continue
        
        # ?destination found
        if current_stop == destination_stop:
            break

        for neighbor_edge in current_stop.get_outgoing_edges():
            
            dep_total = time_to_minutes(neighbor_edge.dep_time)
            arr_total = time_to_minutes(neighbor_edge.arr_time)
            
            if dep_total < current_time: #consider only edges that depart after current time?
                continue
            
            #something could be wrong here - maybe?
            wait_time = dep_total - current_time # if current_stop != starting_stop else 0 #<- better results but RANDOM
            
            #penalties
            new_transfer_count = current_transfers + (1 if (current_line is not None and neighbor_edge.line != current_line) else 0)
    
            # transfer_penalty = 20 if (current_line is not None and neighbor_edge.line != current_line) else 0
            
            if criteria == 't':
                total_edge_cost = (10 if (current_line and neighbor_edge.line != current_line) else 0) + neighbor_edge.travel_time + wait_time
            elif criteria == 'p':
                total_edge_cost = (100*new_transfer_count if (current_line and neighbor_edge.line != current_line) else 0) + wait_time + neighbor_edge.travel_time
            
            new_cost = current_cost + total_edge_cost
            
            # if neighbor_edge.line == current_line and time_to_minutes(neighbor_edge.dep_time) == current_time:
            #     new_cost -= 2
            
            if new_cost < distance[neighbor_edge.end]:
                distance[neighbor_edge.end] = new_cost
                earliest_arrival[neighbor_edge.end] = arr_total
                # transfers[neighbor_edge.end] = new_transfer_count
                previous[neighbor_edge.end] = (current_stop, neighbor_edge, neighbor_edge.line)
                heapq.heappush(priority_queue, (new_transfer_count, new_cost, arr_total, neighbor_edge.end, neighbor_edge.line))
                
    stop_algorithm_time = datetime.now()
    log(f"Dijkstra execution time: {stop_algorithm_time - start_algorithm_time} seconds")
    
    
    path, final_arrival_time = reconstruct_path(previous, starting_stop, destination_stop)
    if path is None:
        print("Error reconstructing path")
        return None, None
    
    total_travel_time = calculate_total_travel_time(start_total, final_arrival_time)
    #print_path(path, starting_stop_name, start_time, total_travel_time)
    
    return path, total_travel_time

## b) Algorytm A*

### Opis teoretyczny metody
Algorytm A* (A-star) to popularny algorytm wyszukiwania ścieżek w grafach, który łączy cechy algorytmu Dijkstry i algorytmu zachłannego przeszukiwania w głąb. Działa na zasadzie minimalizacji funkcji kosztu f(n) = g(n) + h(n), gdzie g to rzeczywisty koszt dojścia ze startu do węzła n, a h to oszacowanie heurystyczne kosztu dojścia z węzła n do celu.

### Przykładowe zastosowania
gry komputerowe, robotyka, automatyzacja, AI

### Wprowadzone modyfikacje
Poniżej wypisano szereg modyfikacji wprowadzonych do algorytmu. Nie wszystkie z nich okazały się działać poprawnie.
- kolejka priorytetowa 
- śledzenie liczby przesiadek oraz wykorzystywanie jej w kolejce priorytetowej
- wprowadzenie heurystyki haversine

### Materiały dodatkowe
- [https://medium.com/@nicholas.w.swift/easy-a-star-pathfinding-7e6689c7f7b2]
- [https://www.redblobgames.com/pathfinding/a-star/introduction.html]

### Wykorzystane biblioteki i funkcje pomocnicze
- datetime do mierzenia czasu
- heapq do kolejki priorytetowej
- math dla heurystyk

In [ ]:
from math import radians, sin, cos, sqrt, atan2

def euclidean_distance(node1, node2):
    return sqrt((node1.lat - node2.lat) ** 2 + (node1.lon - node2.lon) ** 2) * 111000 #deg to m

def haversine_distance(node1, node2):
    R = 6371  #earth radius (km)
    lat1, lon1 = radians(node1.lat), radians(node1.lon)
    lat2, lon2 = radians(node2.lat), radians(node2.lon)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c * 1000  #meters


def manhattan_distance(node1, node2):
    return (abs(node1.lat - node2.lat) + abs(node1.lon - node2.lon)) * 111000 #deg to m


def find_a_star_path(graph, start_name, dest_name, start_time, criteria, heuristic):
    start_algorithm_time = datetime.now()
    start_node = graph.get_node(start_name)
    dest_node = graph.get_node(dest_name)
    
    if start_node is None or dest_node is None:
        print("Starting or destination stop not found")
        return
    
    start_total = time_to_minutes(start_time)
    distance = {node: float('inf') for node in graph.get_nodes()}
    previous = {node: (None, None, None) for node in graph.get_nodes()}
    transfers = {node: float('inf') for node in graph.get_nodes()}
    distance[start_node] = 0
    transfers[start_node] = 0
    
    if heuristic == 'euclidean':
        heuristic_function = lambda node: euclidean_distance(node, dest_node) / 50  # avg speed 50 km/h
    elif heuristic == 'manhattan':
        heuristic_function = lambda node: manhattan_distance(node, dest_node) / 50
    else:
        heuristic_function = lambda node: haversine_distance(node, dest_node) / 50
        
    
    priority_queue = [(0 + heuristic_function(start_node), 0, start_node, None, start_total)]
    
    while priority_queue:
        _, current_cost, current_stop, current_line, current_time = heapq.heappop(priority_queue)
        
        if current_stop == dest_node:
            break
        
        for edge in current_stop.get_outgoing_edges():
            dep_total = time_to_minutes(edge.dep_time)
            if dep_total < current_time:
                continue
            
            wait_time = dep_total - current_time
            #penalties for t/p
            new_transfer_count = transfers[current_stop] + (1 if current_line and edge.line != current_line else 0)
            if criteria == 't':
                total_edge_cost = (10 if (current_line and edge.line != current_line) else 0) + edge.travel_time + wait_time
            elif criteria == 'p':
                total_edge_cost = (100*new_transfer_count if (current_line and edge.line != current_line) else 0) + wait_time + edge.travel_time
            
            #total_edge_cost = edge.travel_time + wait_time + transfer_penalty
            new_cost = current_cost + total_edge_cost
            arr_total = time_to_minutes(edge.arr_time)
            
            #closedList
            if new_cost < distance[edge.end]:
                distance[edge.end] = new_cost
                transfers[edge.end] = new_transfer_count
                previous[edge.end] = (current_stop, edge, edge.line)
                estimated_total_cost = new_cost + heuristic_function(edge.end) #f = g + h
                heapq.heappush(priority_queue, (estimated_total_cost, new_cost, edge.end, edge.line, arr_total))
    
    stop_algorithm_time = datetime.now()
    log(f"A* execution time: {stop_algorithm_time - start_algorithm_time} seconds")
    
    path, final_arrival_time = reconstruct_path(previous, start_node, dest_node)
    total_travel_time = calculate_total_travel_time(start_total, final_arrival_time)
    #print(f'PATH: {path}')
    return path, total_travel_time, distance[dest_node]
    # 


## c) Kryteria t/p

Zgodnie z poleceniem wprowadzono kryteria t oraz p celem wyboru priorytetyzowania albo najkrótszego czasu podróży, albo najmniejszej ilości przesiadek.

In [ ]:
criteria = 'p' #or 't'

if criteria == 't':
                total_edge_cost = (10 if (current_line and neighbor_edge.line != current_line) else 0) + neighbor_edge.travel_time + wait_time
            elif criteria == 'p':
                total_edge_cost = (100*new_transfer_count if (current_line and neighbor_edge.line != current_line) else 0) + wait_time + neighbor_edge.travel_time
                #or:
                total_edge_cost = (math.pow(15,new_transfer_count) if (current_line and neighbor_edge.line != current_line) else 0) + wait_time + neighbor_edge.travel_time
                


## d) Heurystyki

- Euklidesowa - Najkrótsza odległość "po prostej", ignoruje krzywiznę Ziemi.

- Haversine - Dokładna odległość na kuli ziemskiej, uwzględnia jej krzywiznę.

- Manhattan - Ruch po siatce ulic (tylko góra/dół, lewo/prawo), jak w miastach.



In [ ]:
def euclidean_distance(node1, node2):
    return sqrt((node1.lat - node2.lat) ** 2 + (node1.lon - node2.lon) ** 2) * 111000 #deg to m

def haversine_distance(node1, node2):
    R = 6371  #earth radius (km)
    lat1, lon1 = radians(node1.lat), radians(node1.lon)
    lat2, lon2 = radians(node2.lat), radians(node2.lon)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c * 1000  #meters


def manhattan_distance(node1, node2):
    return (abs(node1.lat - node2.lat) + abs(node1.lon - node2.lon)) * 111000 #deg to m

# Zadanie 2. Tabu search

### Opis teoretyczny metody
Tabu search to metaheurystyka optymalizacyjna, która iteracyjnie przeszukuje przestrzeń rozwiązań poprzez lokalne modyfikacje aktualnego rozwiązania. Wykorzystuje listę tabu, aby unikać powtarzania tych samych stanów i zapobiegać utknięciu w minimach lokalnych. Głównym celem jest znalezienie optymalnej permutacji punktów przystankowych, minimalizując koszt podróży.

### Przykładowe zastosowania
optymalizacja tras przejazdu pojazdów transportu publicznego, minimalizacja kosztów logistycznych w planowaniu dostaw, usprawnienie harmonogramowania tras

### Wprowadzone modyfikacje
- zastosowanie wcześniej zaimplementowanego algorytmu A* z heurystyką Manhattan
- Ograniczono rozmiar listy tabu do max(7, len(stop_list)), aby uniknąć nadmiernego wzrostu pamięciowego.
- Wprowadzono maksymalną liczbę iteracji (max_iterations = 100), aby zapewnić zakończenie działania algorytmu.
- Wprowadzono listę tabu w modelu FIFO, aby stopniowo usuwać najstarsze zakazane rozwiązania.


### generate_neighbourhood()

Generuje sąsiedztwo rozwiązania.

In [ ]:
def generate_neighbourhood(stop_list, tabu_list):
    neighbourhood = []
    
    for i in range(len(stop_list)):
        for j in range(i + 1, len(stop_list)):
            new_stop_list = stop_list.copy()
            new_stop_list[i], new_stop_list[j] = new_stop_list[j], new_stop_list[i]  # swap
            
            if tuple(new_stop_list) not in tabu_list:
                neighbourhood.append(new_stop_list)
    
    return neighbourhood

### calculate_cost()

Dla każdej pary przystanków w rozwiązaniu liczy koszty (wykorzystując algorytm A*), a następnie je sumuje.

In [ ]:
def calculate_cost(graph, starting_stop, stop_list, start_time, optimalization_criteria):
    stop_list = [starting_stop] + stop_list + [starting_stop]  #include start/end stop
    total_cost = 0
    total_path = []
    current_start_time = start_time

    for i in range(len(stop_list) - 1):
        path, total_travel_time, route_cost = find_a_star_path(graph, stop_list[i], stop_list[i + 1], current_start_time, optimalization_criteria, 'manhattan')
        current_start_time = minutes_to_time(time_to_minutes(current_start_time) + total_travel_time)
        total_cost += route_cost
        total_path += path

    return total_cost, total_path

Główna implementacja algorytmu tabu:

In [ ]:
def tabu_search(graph, starting_stop, stop_list, start_time, optimalization_criteria):
    current_best_solution = (stop_list, float('inf'), [])
    tabu_list = []
    best_solution = current_best_solution
    
    max_tabu_size = max(7, len(stop_list))  #prevent large factorial growth
    max_iterations = 100
    iteration = 0
    
    while len(tabu_list) < max_tabu_size and iteration < max_iterations:
        neighbourhood = generate_neighbourhood(current_best_solution[0], tabu_list)
        
        if not neighbourhood:  #stop if no new valid swaps exist
            break
        
        costs = {}
        for new_solution in neighbourhood:
            total_cost, total_path = calculate_cost(graph, starting_stop, new_solution, start_time, optimalization_criteria)
            costs[tuple(new_solution)] = [total_cost, total_path]  #store as tuple for performance
        
        best_neighbour = min(costs, key=lambda x: costs[x][0])
        best_neighbour_cost = costs[best_neighbour][0]
        best_neighbour_path = costs[best_neighbour][1]

        current_best_solution = (list(best_neighbour), best_neighbour_cost, best_neighbour_path)

        #fifo
        tabu_list.append(best_neighbour)
        if len(tabu_list) > max_tabu_size:
            tabu_list.pop(0)

        if current_best_solution[1] < best_solution[1]:
            best_solution = current_best_solution
        
        iteration += 1

    return best_solution


## b) rozszerzenie o dobór długości tablicy T w zależności od długości listy L w celu minimalizacji funkcji kosztu

In [ ]:
max_tabu_size = max(5, int(math.sqrt(len(stop_list)) * 2))

## c) o aspirację w celu minimalizacji funkcji kosztu

Aspiracja - pozwala na zaakceptowanie rozwiązania mimo obecności w tablicy tabu

In [ ]:
if tuple(best_neighbour) in tabu_list and best_neighbour_cost >= best_solution[1]:
            continue

SyntaxError: 'continue' not properly in loop (1938951692.py, line 2)

In [ ]:
if tuple(best_neighbour) in tabu_list and best_neighbour_cost < best_solution[1]:
            tabu_list.remove(tuple(best_neighbour))

## d) Strategia próbkowania sąsiedztwa

 Preferujemy ruchy, które przesuwają najdroższe przejścia

In [ ]:
def generate_neighbourhood(stop_list, tabu_list, graph, starting_stop, start_time, optimalization_criteria):
    neighbourhood = []
    expensive_edges = most_expensive_edges(stop_list, graph, starting_stop, start_time, optimalization_criteria)
    
    for _, i, j in expensive_edges:
        new_stop_list = stop_list.copy()
        new_stop_list[i], new_stop_list[j] = new_stop_list[j], new_stop_list[i]
        
        if tuple(new_stop_list) not in tabu_list:
            neighbourhood.append(new_stop_list)
    
    return neighbourhood

def most_expensive_edges(stop_list, graph, starting_stop, start_time, optimalization_criteria):
    costs = []
    current_start_time = start_time
    
    full_stop_list = [starting_stop] + stop_list + [starting_stop]
    
    for i in range(len(full_stop_list) - 1):
        _, travel_time, cost = find_a_star_path(graph, full_stop_list[i], full_stop_list[i + 1], current_start_time, optimalization_criteria, 'manhattan')
        current_start_time = minutes_to_time(time_to_minutes(current_start_time) + travel_time)
        costs.append((cost, i, i + 1))
    
    costs.sort(reverse=True, key=lambda x: x[0])
    return costs[:max(2, len(stop_list)//3)]

Pełna wersja algorytmu

In [ ]:
def generate_neighbourhood(stop_list, tabu_list, graph, starting_stop, start_time, optimalization_criteria):

    neighbourhood = []
    expensive_edges = most_expensive_edges(stop_list, graph, starting_stop, start_time, optimalization_criteria)
    
    for _, i, j in expensive_edges:
        if i >= len(stop_list) or j >= len(stop_list):
            print(f"Warning: Invalid swap indices ({i}, {j}) for stop_list of length {len(stop_list)}")
            continue
        
        new_stop_list = stop_list.copy()
        new_stop_list[i], new_stop_list[j] = new_stop_list[j], new_stop_list[i]  # swap
        
        if tuple(new_stop_list) not in tabu_list:
            neighbourhood.append(new_stop_list)
    
    return neighbourhood


def most_expensive_edges(stop_list, graph, starting_stop, start_time, optimalization_criteria):
    costs = []
    current_start_time = start_time
    
    full_stop_list = [starting_stop] + stop_list + [starting_stop]
    
    for i in range(len(full_stop_list) - 1):
        _, travel_time, cost = find_a_star_path(graph, full_stop_list[i], full_stop_list[i + 1], current_start_time, optimalization_criteria, 'manhattan')
        current_start_time = minutes_to_time(time_to_minutes(current_start_time) + travel_time)
        costs.append((cost, i, i + 1))
    
    costs.sort(reverse=True, key=lambda x: x[0])
    return costs[:max(2, len(stop_list)//3)]


def calculate_cost(graph, starting_stop, stop_list, start_time, optimalization_criteria):
    stop_list = [starting_stop] + stop_list + [starting_stop]  #include start/end stop
    total_cost = 0
    total_path = []
    current_start_time = start_time

    for i in range(len(stop_list) - 1):
        path, total_travel_time, route_cost = find_a_star_path(graph, stop_list[i], stop_list[i + 1], current_start_time, optimalization_criteria, 'manhattan')
        current_start_time = minutes_to_time(time_to_minutes(current_start_time) + total_travel_time)
        total_cost += route_cost
        total_path += path

    return total_cost, total_path

#DOCUMENT THIS
#fix a star finding inconsistent times/solutions
def calculate_cost(graph, starting_stop, stop_list, start_time, optimalization_criteria):
    stop_list = [starting_stop] + stop_list + [starting_stop]
    total_cost = 0
    total_path = []
    current_start_time = start_time

    for i in range(len(stop_list) - 1):
        path, total_travel_time, route_cost = find_a_star_path(graph, stop_list[i], stop_list[i + 1], current_start_time, optimalization_criteria, 'manhattan')
        current_start_time = minutes_to_time(time_to_minutes(current_start_time) + total_travel_time)
        total_cost += route_cost
        total_path += path

    return total_cost, total_path

def tabu_search(graph, starting_stop, stop_list, start_time, optimalization_criteria):
    current_best_solution = (stop_list, float('inf'), [])
    tabu_list = []
    best_solution = current_best_solution
    
    max_tabu_size = max(5, int(math.sqrt(len(stop_list)) * 2))
    max_iterations = 100
    iteration = 0
    
    while len(tabu_list) < max_tabu_size and iteration < max_iterations:
        neighbourhood = generate_neighbourhood(current_best_solution[0], tabu_list, graph, starting_stop, start_time, optimalization_criteria)
        
        if not neighbourhood:
            break
        
        costs = {}
        for new_solution in neighbourhood:
            total_cost, total_path = calculate_cost(graph, starting_stop, new_solution, start_time, optimalization_criteria)
            costs[tuple(new_solution)] = [total_cost, total_path]
        
        best_neighbour = min(costs, key=lambda x: costs[x][0])
        best_neighbour_cost = costs[best_neighbour][0]
        best_neighbour_path = costs[best_neighbour][1]
        
        if tuple(best_neighbour) in tabu_list and best_neighbour_cost >= best_solution[1]:
            continue
        
        if tuple(best_neighbour) in tabu_list and best_neighbour_cost < best_solution[1]:
            tabu_list.remove(tuple(best_neighbour))
        
        current_best_solution = (list(best_neighbour), best_neighbour_cost, best_neighbour_path)
        tabu_list.append(best_neighbour)
        
        if len(tabu_list) > max_tabu_size:
            tabu_list.pop(0)  #fIFO
        
        if current_best_solution[1] < best_solution[1]:
            best_solution = current_best_solution
        
        iteration += 1
    
    return best_solution

# Wyniki, wnioski

## Tabu: PL. GRUNWALDZKI, Wrocławski Park Przemysłowy;DWORZEC GŁÓWNY, 14:40, p

```
Shortest path from PL. GRUNWALDZKI at 14:40:
Start at PL. GRUNWALDZKI (Time: 14:40)
  → Change to line 4 at PL. GRUNWALDZKI
    → Depart at 14:40:00 from PL. GRUNWALDZKI
    → Arrive at most Grunwaldzki at 14:41:00 (1 mins)
    → Depart at 14:41:00 from most Grunwaldzki
    → Arrive at Urząd Wojewódzki (Impart) at 14:43:00 (2 mins)
    → Depart at 14:43:00 from Urząd Wojewódzki (Impart)
    → Arrive at pl. Wróblewskiego at 14:46:00 (3 mins)
    → Depart at 14:46:00 from pl. Wróblewskiego
    → Arrive at Komuny Paryskiej at 14:47:00 (1 mins)
    → Depart at 14:47:00 from Komuny Paryskiej
    → Arrive at Kościuszki at 14:48:00 (1 mins)
    → Depart at 14:48:00 from Kościuszki
    → Arrive at Pułaskiego at 14:50:00 (2 mins)
    → Depart at 14:50:00 from Pułaskiego
    → Arrive at DWORZEC GŁÓWNY at 14:52:00 (2 mins)
    → Depart at 14:52:00 from DWORZEC GŁÓWNY
    → Arrive at Arkady (Capitol) at 14:55:00 (3 mins)
    → Depart at 14:55:00 from Arkady (Capitol)
    → Arrive at pl. Legionów at 14:57:00 (2 mins)
  → Change to line 21 at pl. Legionów
    → Depart at 14:58:00 from pl. Legionów
    → Arrive at pl. Orląt Lwowskich at 15:00:00 (2 mins)
    → Depart at 15:00:00 from pl. Orląt Lwowskich
    → Arrive at PL. JANA PAWŁA II at 15:03:00 (3 mins)
    → Depart at 15:03:00 from PL. JANA PAWŁA II
    → Arrive at Młodych Techników at 15:05:00 (2 mins)
    → Depart at 15:05:00 from Młodych Techników
    → Arrive at pl. Strzegomski (Muzeum Współczesne) at 15:06:00 (1 mins)
  → Change to line 13 at pl. Strzegomski (Muzeum Współczesne)
    → Depart at 15:13:00 from pl. Strzegomski (Muzeum Współczesne)
    → Arrive at Dolmed at 15:15:00 (2 mins)
    → Depart at 15:15:00 from Dolmed
    → Arrive at Śrubowa at 15:16:00 (1 mins)
    → Depart at 15:16:00 from Śrubowa
    → Arrive at Wrocławski Park Przemysłowy at 15:17:00 (1 mins)
  → Change to line 148 at Wrocławski Park Przemysłowy
    → Depart at 15:17:00 from Wrocławski Park Przemysłowy
    → Arrive at Śrubowa at 15:19:00 (2 mins)
    → Depart at 15:19:00 from Śrubowa
    → Arrive at Smolecka at 15:20:00 (1 mins)
    → Depart at 15:20:00 from Smolecka
    → Arrive at Dworzec Świebodzki at 15:22:00 (2 mins)
    → Depart at 15:22:00 from Dworzec Świebodzki
    → Arrive at pl. Orląt Lwowskich at 15:24:00 (2 mins)
    → Depart at 15:24:00 from pl. Orląt Lwowskich
    → Arrive at Renoma at 15:26:00 (2 mins)
    → Depart at 15:26:00 from Renoma
    → Arrive at Dworzec Główny (Dworcowa) at 15:29:00 (3 mins)
  → Change to line A at Dworzec Główny (Dworcowa)
    → Depart at 15:29:00 from Dworzec Główny (Dworcowa)
    → Arrive at skwer Krasińskiego at 15:31:00 (2 mins)
    → Depart at 15:31:00 from skwer Krasińskiego
    → Arrive at Krasińskiego at 15:32:00 (1 mins)
    → Depart at 15:32:00 from Krasińskiego
    → Arrive at Urząd Wojewódzki (Muzeum Narodowe) at 15:35:00 (3 mins)
    → Depart at 15:35:00 from Urząd Wojewódzki (Muzeum Narodowe)
    → Arrive at Katedra at 15:36:00 (1 mins)
  → Change to line 10 at Katedra
    → Depart at 15:38:00 from Katedra
    → Arrive at Reja at 15:40:00 (2 mins)
    → Depart at 15:40:00 from Reja
    → Arrive at PL. GRUNWALDZKI at 15:43:00 (3 mins)

Total travel time: 63 minutes
```


## Dijkstra - Kochanowskiego->Rynek, 22:30, t

```
Shortest path from Kochanowskiego at 22:30:
Start at Kochanowskiego (Time: 22:30)
  → Change to line 13 at Kochanowskiego
    → Depart at 22:32:00 from Kochanowskiego
    → Arrive at Bujwida at 22:34:00 (2 mins)
    → Depart at 22:34:00 from Bujwida
    → Arrive at PL. GRUNWALDZKI at 22:36:00 (2 mins)
    → Depart at 22:36:00 from PL. GRUNWALDZKI
    → Arrive at most Grunwaldzki at 22:37:00 (1 mins)
    → Depart at 22:37:00 from most Grunwaldzki
    → Arrive at Urząd Wojewódzki (Impart) at 22:39:00 (2 mins)
    → Depart at 22:39:00 from Urząd Wojewódzki (Impart)
    → Arrive at GALERIA DOMINIKAŃSKA at 22:42:00 (3 mins)
    → Depart at 22:42:00 from GALERIA DOMINIKAŃSKA
    → Arrive at Świdnicka at 22:44:00 (2 mins)
    → Depart at 22:44:00 from Świdnicka
    → Arrive at Zamkowa at 22:45:00 (1 mins)
    → Depart at 22:45:00 from Zamkowa
    → Arrive at Rynek at 22:47:00 (2 mins)

Total travel time: 17 minutes
```


## A* - Kliniki - Politechnika Wrocławska->Grabiszyńska 17:30, p, manhattan

```
Shortest path from Kliniki - Politechnika Wrocławska at 17:30:
Start at Kliniki - Politechnika Wrocławska (Time: 17:30)
  → Change to line 10 at Kliniki - Politechnika Wrocławska
    → Depart at 17:30:00 from Kliniki - Politechnika Wrocławska
    → Arrive at PL. GRUNWALDZKI at 17:32:00 (2 mins)
    → Depart at 17:32:00 from PL. GRUNWALDZKI
    → Arrive at Reja at 17:33:00 (1 mins)
    → Depart at 17:33:00 from Reja
    → Arrive at Katedra at 17:35:00 (2 mins)
    → Depart at 17:35:00 from Katedra
    → Arrive at Urząd Wojewódzki (Muzeum Narodowe) at 17:37:00 (2 mins)
    → Depart at 17:37:00 from Urząd Wojewódzki (Muzeum Narodowe)
    → Arrive at GALERIA DOMINIKAŃSKA at 17:40:00 (3 mins)
    → Depart at 17:40:00 from GALERIA DOMINIKAŃSKA
    → Arrive at Świdnicka at 17:42:00 (2 mins)
    → Depart at 17:42:00 from Świdnicka
    → Arrive at Zamkowa at 17:43:00 (1 mins)
  → Change to line 11 at Zamkowa
    → Depart at 17:43:00 from Zamkowa
    → Arrive at Narodowe Forum Muzyki at 17:45:00 (2 mins)
    → Depart at 17:45:00 from Narodowe Forum Muzyki
    → Arrive at pl. Legionów at 17:47:00 (2 mins)
    → Depart at 17:47:00 from pl. Legionów
    → Arrive at Kolejowa at 17:48:00 (1 mins)
    → Depart at 17:48:00 from Kolejowa
    → Arrive at Grabiszyńska at 17:49:00 (1 mins)

Total travel time: 19 minutes
```


Zadanie byłob bardzo czasochłonne, ale też i niezwykle ciekawe. Podczas implementacji zostało przetestowane wiele różnych modyfikacji oraz heurystyk. Nie wszystkie działają idealnie, ale niektóre wielce usprawniły działanie algorytmów, nawet o kilka sekund. Ustalenie trasy nie było najtrudniejszym zadaniem. Największym wyzwaniem było wyznaczenie jej tak, aby była optymalna pod kątem przesiadek. 